## 환경 확인
- GPU 사용 가능 여부, GPU 종류, CUDA 버전 출력
- CUDA는 NVIDIA에서 만든 GPU 병렬 연산을 위한 플랫폼 및 API (CPU 대신 GPU)

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

True
NVIDIA GeForce RTX 3060 Ti
12.1


## 모델 불러오기
- **YOLOv8n (nano)**: 가장 작은 모델, 가볍고 빠르지만 정확도 낮음 → 임베디드 장치나 실시간성 최우선 환경에 적합
- **YOLOv8s (small)**: 작은 모델, 속도와 정확도의 균형이 괜찮음 → 데이터가 많지 않은 경우, 보통 실험 시작 시점에 많이 사용
- **YOLOv8m (medium)**: 중간 크기, 정확도가 올라가지만 학습 속도·메모리 소모도 커짐
- **YOLOv8l (large)**: 큰 모델, 정확도는 높지만 GPU 자원과 시간이 많이 필요 → 데이터셋이 충분히 크고, 성능이 최우선일 때 선택
- **YOLOv8x (extra large)**: 가장 큰 모델, 최고 성능이지만 자원 소모가 큼

- **s모델을 사용한 이유**: 데이터셋 크기가 아직 크지 않았기 떄문에 s 모델이 과적합 위험을 줄이고 빠른 학습 실험에 적합하다고 판단

### 모델 학습
- 데이터셋 정보가 담긴 yaml 파일을 불러와 30 epoch 동안 학습

- 이미지 크기 640, 배치 사이즈 8, GPU에서 학습 수행

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt") 
results = model.train(
    data="C:/Users/justi/Desktop/yolo_data/yaml.yaml",
    epochs=30,          # 학습 반복 횟수
    imgsz=640,          # 입력 이미지 크기 (값이 클수록 높은 해상도 학습, GPU 메모리 사용량 ↑)
    batch=8,            # 한 번에 학습하는 이미지 수 (값이 클수록 학습이 안정적, GPU 연산 효율 ↑, 메모리 부족)
    device=0,           # GPU 디바이스 ID (0번 GPU 사용)
)

New https://pypi.org/project/ultralytics/8.3.203 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.199  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/justi/Desktop/yolo_data/yaml.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=6

### 모델 검증 (Validation)

- 학습된 모델을 validation set으로 평가

- Precision, Recall, mAP 등 주요 성능 지표 산출 (정량적 지표)

In [3]:
val_results = model.val(
    data="C:/Users/justi/Desktop/yolo_data/yaml.yaml",
    imgsz=640,
    device=0
)
val_results

Ultralytics 8.3.199  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 66.969.9 MB/s, size: 129.7 KB)
val: Scanning C:\Users\justi\Desktop\yolo_data\labels\val.cache... 119 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 119/119 118.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 15/15 1.5it/s 10.1s<0.1s
                   all        119        122      0.967      0.951      0.985      0.775
Speed: 0.3ms preprocess, 7.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to C:\Users\justi\Desktop\runs\detect\train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000026B767540D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

### 모델 검증 2 (Validation)

- 시각적 검증을 위한 절차 (시각적 결과)

In [ ]:
pred = model.predict(
    source="C:/Users/justi/Desktop/yolo_data/images/val",  
    imgsz=640,
    conf=0.25, # 얼만큼 확신하면 결과로 인정할까 (처음에는 조금 낮게 데이터 많아지면 점점 올리기)
    save=True
)


image 1/119 C:\Users\justi\Desktop\yolo_data\images\val\Img_050_0845.jpg: 640x448 1 food, 54.1ms
image 2/119 C:\Users\justi\Desktop\yolo_data\images\val\Img_050_0859.jpg: 512x640 1 food, 53.9ms
image 3/119 C:\Users\justi\Desktop\yolo_data\images\val\Img_050_0861.jpg: 448x640 1 food, 51.5ms
image 4/119 C:\Users\justi\Desktop\yolo_data\images\val\Img_050_0862.jpg: 448x640 1 food, 8.0ms
image 5/119 C:\Users\justi\Desktop\yolo_data\images\val\Img_050_0865.jpg: 448x640 1 food, 7.8ms
image 6/119 C:\Users\justi\Desktop\yolo_data\images\val\Img_050_0871.jpg: 512x640 1 food, 8.2ms
image 7/119 C:\Users\justi\Desktop\yolo_data\images\val\Img_050_0873.jpg: 544x640 1 food, 50.9ms
image 8/119 C:\Users\justi\Desktop\yolo_data\images\val\Img_050_0874.jpg: 640x640 1 food, 9.6ms
image 9/119 C:\Users\justi\Desktop\yolo_data\images\val\Img_050_0877.jpg: 384x640 1 food, 51.7ms
image 10/119 C:\Users\justi\Desktop\yolo_data\images\val\Img_050_0878.jpg: 384x640 1 food, 8.2ms
image 11/119 C:\Users\justi\Deskt